In [1]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [2]:
from ipywidgets import widgets
from IPython.display import display, IFrame, HTML, Audio
import ipywidgets
import functools
import os
import random
from IPython.display import clear_output
from IPython.display import FileLink
from ipywidgets import Layout, Button, Box, VBox
class Character:
    def __init__(self, hp, dc, img_path, attack, dmg, special):
        self.hp = hp
        self.dc = dc
        self.img = img_path
        self.attack = attack
        self.dmg = dmg
        self.special = special
    @staticmethod
    def shield():
        self.dc = self.dc + 2
    @staticmethod
    def sword():
        self.attack = self.attack + 2
    @staticmethod
    def fire_sword():
        self.dmg = self.dmg + 2
    @staticmethod
    def heal():
        self.hp = self.hp + 4

druid = Character(4, 4, os.path.join(f"resources", "Character_selection","druid.png"), 2, 0, 0)
wizard = Character(4, 4, os.path.join(f"resources", "Character_selection","wizard.png"), 2, 0, 0)
rouge = Character(4, 4, os.path.join(f"resources", "Character_selection","rouge.png"), 2, 0, 0)
angel = Character(4, 4, os.path.join(f"resources", "Character_selection","angel.png"), 2, 0, 0)
dragon = Character(4, 4, os.path.join(f"resources", "Character_selection","dragon.png"), 2, 0, 0)
warrior = Character(4, 4, os.path.join(f"resources", "Character_selection","warrior.png"), 2, 0, 0)
monster = Character(8, 4, os.path.join(f"resources", "characters","monster.png"), 3, 0, 0)

characters = {"druid":druid, "wizard":wizard, "rouge":rouge, "angel":angel, "dragon":dragon, "warroir":warrior}
python= 1
intro = 0


def get_python():
    try:
        global python
        file = open(os.path.join(f"resources", "story", f"{python}.png"), "rb")
        python = python + 1
    except OSError:
        print(python, "the end")
        
    image = file.read()
    return widgets.Image(
        value=image,
        #width=500,
        #height=600,
    )
def get_intro():
    global intro
    file = open(os.path.join(f"resources", "intro", f"{intro}.png"), "rb")
    intro = intro + 1

    image = file.read()
    return widgets.Image(
        value=image,
        #width=500,
        #height=600,
    )

def next_intro(b):
    global intro
    if intro == 2:
        choose_characters()
    else:
        run_intro()

def back_page(b):
    global python
    python = python-2
    run()


def next_page(b):
    can_pass_to_next_report = True#check_mandatory_fields(subj_results, task)
    if can_pass_to_next_report:
     
        run()
    else:
        b.button_style='warning'
        
def roll_dice(b, lbl=None):
    lbl.value = str(random.randrange(1,7))

def get_choises(characters):
    return [widgets.Label("choose character:")]+[widgets.GridBox([widgets.Checkbox(description= i) for i in 
        characters.keys()])]

def choose_func(b, chars=None, targets_buttons=None):
    chosen = {i.description:i.value for i in targets_buttons[1].children}
    global characters
    characters = {i: characters[i] for i in characters.keys() if chosen[i]}
    run_story()

def show_characters(b, characters=characters):
    wids = []

    for char in characters.keys():
        file = open(characters[char].img, "rb")
        image = file.read()
        img = widgets.Image(
            value=image,
            width=500,
            height=600,
        )
        lbl = widgets.Label(f"{char}, hp = {characters[char].hp}")
        wids.append(widgets.GridBox([lbl, img]))
    display(widgets.GridBox([*wids]))


def get_targets(characters):
    return widgets.ToggleButtons(options=characters.keys())     

def hit_func(b, targets_buttons=None, characters=characters, dmg=1):
    target = characters[targets_buttons.value]
    target.damage(dmg)
    print(target)
    play_turn()
    
def miss_func(b):
    play_turn()
    
def attack(b, attacker=None, lbl=None, hit=None, miss=None, targets_buttons=None, characters=characters):
    target = characters[targets_buttons.value]
    if target is None:
        b.button_style='warning'
    else:
        dice =  random.randrange(1,7)
        atk = attacker.attack + dice
        
        lbl.value = f"dice = {dice}\nattack = {atk}\ndef {target.dc}"
        hit.disabled = False
        miss.disabled = False
## screens    

def choose_characters():
    clear_output()
    targets_buttons = get_choises(characters)
    choose = widgets.Button(description=f"choose")
    choose.on_click(functools.partial(choose_func, chars=characters, targets_buttons=targets_buttons))
    chars = widgets.Button(description=f"show Characters")
    chars.on_click(functools.partial(show_characters, characters=characters))
    
    display(widgets.GridBox([*targets_buttons, choose, chars]))
    
    
def run_story():
    clear_output()
    back_button = widgets.Button(description=f"back")
    back_button.on_click(functools.partial(back_page))
    
    next_button = widgets.Button(description=f"next")
    next_button.on_click(functools.partial(next_page))
    
    dice_lbl = widgets.Label("0")
    dice = widgets.Button(description=f"roll dice")
    dice.on_click(functools.partial(roll_dice, lbl=dice_lbl))
    
    chars = widgets.Button(description=f"show Characters")
    chars.on_click(functools.partial(show_characters, characters=characters))
    if python == 1:
        img = get_python()
        display(widgets.GridBox([img, next_button, dice, dice_lbl, chars]))
    else:
        img = get_python()
        display(widgets.GridBox([img, widgets.Box([back_button, next_button, dice, dice_lbl, chars])]))

def play_turn():
    global battle_round
    try:
        turn = next(battle_round)
        battle(turn, characters)
    except StopIteration:
        battle_round = iter([*characters.keys()])
        play_turn()

def battle(turn, characters):
    clear_output()
    turn_lbl = widgets.Label(turn)
        
    next_button = widgets.Button(description=f"finish battle")
    next_button.on_click(functools.partial(next_page))

    attacker = None
    target = None
    targets_buttons = get_targets(characters)
    hit = ipywidgets.Button(description="hit", disabled=True)
    hit.on_click(functools.partial(hit_func, targets_buttons=targets_buttons, characters=characters))

    miss = ipywidgets.Button(description="miss", disabled=True)
    miss.on_click(functools.partial(miss_func))
    attack_lbl = widgets.Label("")
    attack_button = widgets.Button(description=f"roll dice")
    
    chars = widgets.Button(description=f"show Characters")
    chars.on_click(functools.partial(show_characters, characters=characters))
    if turn == "monster":
        attack_button.on_click(functools.partial(attack, attacker=monster, lbl=attack_lbl, hit=hit, miss=miss, targets_buttons=targets_buttons, characters=characters))
        display(widgets.GridBox([next_button, turn_lbl, targets_buttons, attack_lbl,attack_button, hit, miss, chars]))
        
        
    else: 
        attacker = characters[turn]
        targets_buttons.value = "monster"
        attack_button.on_click(functools.partial(attack, attacker=attacker, lbl=attack_lbl, hit=hit, miss=miss, targets_buttons=targets_buttons, characters=characters))
        display(widgets.GridBox([next_button, turn_lbl, attack_lbl,attack_button, hit, miss, chars]))
        
def run_intro():
    clear_output()

    next_button = widgets.Button(description=f"next")
    next_button.on_click(functools.partial(next_intro))
    img = get_intro()
    display(widgets.GridBox([img, next_button]))
run_intro()    

##
#battle_round = iter([*characters.keys()])

#play_turn()

GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xb0\x00\x00\x02\x12\x08\x02\x0…